In [ ]:
class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.edges = []
    
    def add_edge(self, u, v, weight):
        self.edges.append((u, v, weight))

    def bellman_ford(self, source):
        distance = [float('inf')] * self.vertices
        distance[source] = 0

        # relaxation
        for _ in range(self.vertices - 1):
            for u, v, weight in self.edges:
                if distance[u] != float('inf') and distance[u] + weight < distance[v]:
                    distance[v] = distance[u] + weight
        
        # detection
        for u, v, weight in self.edges:
            if distance[u] != float('inf') and distance[u] + weight < distance[v]:
                return "Graph contains a negative-weight cycle"

        return distance


        

In [15]:
distance = [float('inf')] * 5
distance

[inf, inf, inf, inf, inf]

In [23]:
import heapq
import math

def bellman_ford(graph, source, n): # 
    dist = [math.inf] * n
    dist[source] = 0
    
    for _ in range(n - 1): # relaxed these times casue if there at lease only one node connected to all verticess will be relaxxed the same times
        for u in range(n):
            for v,w in graph[u]: # if fully connected graph this alg will be o(v^3)
                if dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
    for u in range(n):
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                raise ValueError("Graph contains a negative weight cycle")
    return dist

def dijkstra(graph, source, n):
    dist = [math.inf] * n
    dist[source] = 0
    pq = [(0, source)]

    while pq:# will use eager dijkstra
        d, u = heapq.heappop(pq)
        if d > dist[u]:
            continue
        for v, w in graph[u]:
            if dist[u] + w  < dist[v]:
                dist[v] = dist[u] + w
                heapq.heappush(pq, (dist[v], v))
    return dist


def johnson(graph, n):
    modified_graph = graph + [[]]  # Add a new vertex with edges to all others
    for i in range(n):
        modified_graph[-1].append((i, 0))
    h = bellman_ford(modified_graph, n, n+1) # n is start from jonson 0 vertex
    reweighted_graph = [[] for _ in range(n)]
    for u in range(n):
        for v, w in graph[u]:
            reweighted_graph[u].append((v, w + h[u] - h[v]))
    all_pairs_shortest_paths = []
    for i in range(n):
        dist = dijkstra(reweighted_graph, i, n)
        for j in range(n):
            if dist[j] < math.inf: # path exsit
                dist[j] += h[j] - h[i]
        all_pairs_shortest_paths.append(dist)

    return all_pairs_shortest_paths  # Return the matrix of all-pairs shortest paths

n = 5
graph = [
    [(1, 3), (2, 8)],    # Vertex 0: edges to 1 with weight 3, 2 with weight 8
    [(2, 2), (3, 1)],    # Vertex 1: edges to 2 with weight 2, 3 with weight 1
    [(4, 4)],            # Vertex 2: edge to 4 with weight 4
    [(2, -5), (4, 2)],   # Vertex 3: edge to 2 with weight -5, 4 with weight 2
    []                   # Vertex 4: no outgoing edges
]


try:
    result = johnson(graph, n)  # Call the Johnson's algorithm to compute all-pairs shortest paths
    for i in range(n):
        print(f"Shortest paths from node {i}: {result[i]}")  # Print shortest paths from each node
except ValueError as e:
    print(e)  # Print error message if a negative weight cycle is detected
    

Shortest paths from node 0: [0, 3, -1, 4, 3]
Shortest paths from node 1: [inf, 0, -4, 1, 0]
Shortest paths from node 2: [inf, inf, 0, inf, 4]
Shortest paths from node 3: [inf, inf, -5, 0, -1]
Shortest paths from node 4: [inf, inf, inf, inf, 0]


In [19]:
graph = [
    [(1, 3), (2, 8)],    # Vertex 0: edges to 1 with weight 3, 2 with weight 8
    [(2, 2), (3, 1)],    # Vertex 1: edges to 2 with weight 2, 3 with weight 1
    [(4, 4)],            # Vertex 2: edge to 4 with weight 4
    [(2, -5), (4, 2)],   # Vertex 3: edge to 2 with weight -5, 4 with weight 2
    []                   # Vertex 4: no outgoing edges
]

a= [[(i, 0) for i in range(5)]]
a, graph + a

([[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]],
 [[(1, 3), (2, 8)],
  [(2, 2), (3, 1)],
  [(4, 4)],
  [(2, -5), (4, 2)],
  [],
  [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]])

In [ ]:
def floyd_warshall(graph):
    v = len(graph)
    dist = [[float('inf')] * v for _ in range(v)]

    for i in range(v):
        for j in range(v):
            if i == j:
                dist[i][j] = 0
            elif graph[i][j] != 0:
                dist[i][j] = graph[i][j]
    
    for k in range(v):
        for i in range(v):
            for j in range(v):
                dist[i][j] = min(dist[i][j] , dist[i][k] + dist[k][j])
    return dist


In [24]:
import sys

def tsp(n, dist):
    dp = [[sys.maxsize] * n for _ in range(1 << n)]
    """
sys.maxsize is part of the sys module and is typically used to represent the 
maximum size of a container, like a list or an array. It’s also commonly used 
as an indicator of infinity when implementing algorithms that need a large 
placeholder value (like in Dynamic Programming, where it might represent an 
initially "infinite" cost or distance).    
    """
    # Starting at city 0
    dp[1][0] = 0
    
    for mask in range(1 << n):  # 1 << n is 2^n, representing all subsets of cities
        for u in range(n):
            if (mask & (1 << u)) == 0:
                continue
            for v in range(n):
                if mask & (1 << v):
                    continue
                next_mask = mask | (1 << v)
                # Update the DP table with the minimum cost for visiting cities in next_mask, 
                  # ending at city 'v'
                dp[next_mask][v] = min(dp[next_mask][v], dp[mask][u] + dist[u][v])
    
    final_cost = sys.maxsize
    for i in range(1, n): 

        final_cost = min(final_cost, dp[(1 << n) - 1][i] + dist[i][0])

    return final_cost, dp


In [ ]:
from collections import defaultdict

class EularianGraph:
    def __init__(self, graph):
        self.graph = graph
        self.in_degree = defaultdict{int}
        self.out_degree = defaultdict{int}
        self.edge_count = 0
        self.initialize_degrees()

    def _initilize_degrees(self):
        for node in self.graph:
            for neighbor in self.graph[node]:
                self.in_degree[neighbor] += 1
                self.out_degree[neighbor] += 1
                self.edge_count += 1
    
    def find_start_node(self):
        start_nodes, end_nodes, start_node = 0, 0, None

        for node in self.graph:
            out_diff_in  = self.out_degree[node] - self.in_degree[node]
            in_diff_out  = self.in_degree[node]  - self.out_degree[node]

            if abs(out_diff_in) > 1 or abs(in_diff_out) > 1:
                return None 
            
            if out_diff_in == 1:
                start_nodes += 1
                start_node = node
            elif in_diff_out == 1:
                end_nodes += 1
            
        # it could be eular circuit so, 
        if start_nodes == 0 and end_nodes == 0:
            for node in self.graph:
                if self.out_degree[node] > 0:
                    return node
        elif start_nodes == 1 and end_nodes == 1:
            return start_node
        return None
    
    def eulerian_parth_or_circuit(self):
        start_node = self.find_start_node()
        if start_node is None:
            return None
        
        path = []
        self._dfs(start_node, path)

        if len(path) == self.edge_count + 1:
            return path
        else:
            return None
        
    def _dfs(self, node, path):
        while self.out_degree[node] > 0:
            next_node = self.graph[node].pop()
            self.out_degree[node] -= 1
            self._dfs(next_node, path)
        path.append(node)


In [ ]:
import heapq

def laze_prim(graph, start):
    mst = []
    visited = set()
    min_heap = [(0, start)]
    while min_heap:
        weight, vertex = heapq.heappop(min_heap)
        if vertex not  in visited:
            visited.add(vertex)
            if weight > 0:
                mst.append((weight, vertex))
            for neighbor, edge_weight in graph[vertex]:
                if neighbor not in visited:
                    heapq.heappush(min_heap, (edge_weight, neighbor))
    return mst

In [ ]:
import heapq

def prim(graph, start):
    mst = [] 
    visited = set()  
    min_heap = [(0, start)]  

    while min_heap:
        weight, vertex = heapq.heappop(min_heap)
        
        if vertex not in visited:
            visited.add(vertex)
            if weight > 0: 
                mst.append((weight, vertex))

            for neighbor, edge_weight in graph[vertex]:
                if neighbor not in visited:
                    heapq.heappush(min_heap, (edge_weight, neighbor))
            visited.add(vertex)

    return mst


In [ ]:
def eager_prim(graph, start):
    mst = []
    visited = set()
    min_heap = []

    def add_edges(node):
        visited.add(node)
        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                heapq.heappush(min_heap,(weight, node, neighbor))
    add_edges(start)

    while min_heap:
        weight, u, v = heapq.heappop(min_heap)

        if v not in visited:
            mst.append((u, v, weight))
            add_edges(v)
    return mst
    

In [ ]:
####################################################################################################
def initialize_residual_graph_ford_fullkerson(graph):
    residual_graph = {u: dict(graph[u]) for u in graph} # copy from original graph
    # print("Before looping --> residual_graph ", residual_graph)
    for u in graph:
        for v in graph[u]:
            if v not in residual_graph:
                residual_graph[v] = {}
            if u not in residual_graph[v]:
                residual_graph[v][u] = 0
    # print("After init residual_graph and become directed for both directions: ", residual_graph)                 
    return residual_graph

####################################################################################################

from collections import deque, defaultdict
def initialize_residual_graph_Edmos(graph):
    residual_graph = defaultdict(dict) # defaultdict(dict, {})
    for u in graph:
        for v, capacity in graph[u].items():
            residual_graph[u][v] = capacity
            if v not in residual_graph:
                residual_graph[v] = {}
            if u not in residual_graph[v]:
                residual_graph[v][u] = 0
    return residual_graph

####################################################################################################


####################################################################################################
graph = {
    0: {1: 10, 2: 10},     # Source node 0 has edges to 1 and 2
    1: {3: 10},            # Node 1 has an edge to 3
    2: {3: 10},            # Node 2 has an edge to 3
    3: {4: 10},            # Sink node 4 has an edge from 3
    4: {}                  # No outgoing edges from sink
}
residual_graph = initialize_residual_graph_ford_fullkerson(graph)








Before looping --> residual_graph  {0: {1: 10, 2: 10}, 1: {3: 10}, 2: {3: 10}, 3: {4: 10}, 4: {}}
After init residual_graph and become directed for both directions:  {0: {1: 10, 2: 10}, 1: {3: 10, 0: 0}, 2: {3: 10, 0: 0}, 3: {4: 10, 1: 0, 2: 0}, 4: {3: 0}}


In [53]:
def dfs_ford_fullkerson(graph, source, sink):
    visited = set()
    parent = {}

    def dfs_visit(node):
        if node == sink:
            return True
        for neighbor in graph[node]:
            if neighbor not in visited and graph[node][neighbor] > 0:
                visited.add(neighbor)
                parent[neighbor] = node
                if dfs_visit(neighbor):
                    return True
        return False
    visited.add(source)
    if dfs_visit(source): # enter when all cales are True && True && True ....
        return parent
    return None
####################################################################################################
from collections import deque

def bfs_Edmonds(residual_graph,source, sink):
    visited = set()
    parent = {}

    queue = deque([source])
    visited.add(source)

    while queue:
        node = queue.popleft()
        for neighbor, capacity in residual_graph[node].items():
            if neighbor not in visited and capacity > 0:
                parent[neighbor] = node
                visited.add(neighbor)
                if neighbor == sink:
                    return parent
                queue.append(neighbor)
    return None

####################################################################################################
parent = dfs_ford_fullkerson(residual_graph, 0, 4)
parent

{1: 0, 3: 1, 4: 3}

In [54]:
def ford_fullkerson(graph, source, sink):
    flow = 0
    residual_graph = initialize_residual_graph_ford_fullkerson(graph)

    while True:
        # finding path consists of edges in the residual graph that have a positive residual capacity.
        parent = dfs_ford_fullkerson(residual_graph, source, sink)
        if parent is None:
            break

        # Min Capacity 
        path_flow = float('inf')
        current = sink
        while current != source:
            # {1: 0, 3: 1, 4: 3} --> parent[current] = 3 at sink
                # residual_graph[parent[current]][current] = 10 at sink
            path_flow = min(path_flow, residual_graph[parent[current]][current])
            current = parent[current]
        
        # augmenting flow 
        current = sink
        while current != source:
            prev = parent[current]
            residual_graph[prev][current] -= path_flow
            residual_graph[current][prev] += path_flow
            current = prev
        
        flow += path_flow

    return flow 
####################################################################################################
def ford_fulkerson_Edmonds(graph, source, sink):
    residual_graph = initialize_residual_graph_Edmos(graph)
    max_flow = 0 
    while True:
        parent = bfs_Edmonds(residual_graph,source, sink)
        if parent is None:
            break

        path_flow = float('inf')
        current = sink
        while current != source:
            prev = parent[current]
            path_flow = min(path_flow, residual_graph[prev][current])
            current = prev
        
        current = sink
        while current != source:
            prev = parent[current]
            residual_graph[prev][current] -= path_flow
            residual_graph[current][prev] += path_flow
            current = prev
        max_flow += path_flow

    return max_flow        

####################################################################################################

# Example usage:
graph = {
    0: {1: 10, 2: 10},     # Source node 0 has edges to 1 and 2
    1: {3: 10},            # Node 1 has an edge to 3
    2: {3: 10},            # Node 2 has an edge to 3
    3: {4: 10},            # Sink node 4 has an edge from 3
    4: {}                  # No outgoing edges from sink
}

source = 0
sink = 4

max_flow = ford_fullkerson(graph, source, sink)
print(f"Maximum Flow: {max_flow}")

####################################################################################################
# Example usage
if __name__ == "__main__":
    # Example graph as adjacency list
    graph = {
        's': {'a': 10, 'b': 5},
        'a': {'b': 15, 't': 10},
        'b': {'t': 10},
        't': {}
    }
    source = 's'
    sink = 't'
    max_flow = ford_fulkerson_Edmonds(graph, source, sink)
    print(f"Maximum flow: {max_flow}")



Before looping --> residual_graph  {0: {1: 10, 2: 10}, 1: {3: 10}, 2: {3: 10}, 3: {4: 10}, 4: {}}
After init residual_graph and become directed for both directions:  {0: {1: 10, 2: 10}, 1: {3: 10, 0: 0}, 2: {3: 10, 0: 0}, 3: {4: 10, 1: 0, 2: 0}, 4: {3: 0}}
Maximum Flow: 10
Maximum flow: 15


In [55]:
from collections import deque, defaultdict
from typing import List, Optional

class Dinic:
    def __init__(self, n):
        self.n = n
        self.graph = [[] for _ in range(n)]
        self.level = [-1] * n   # level init by -1 
        self.ptr = [0] * n # Pointer for each node to track DFS progress

    class Edge:
        def __init__(self, u: int, v: int, capacity: int, flow: int = 0):
            self.u = u
            self.v = v
            self.capacity = capacity
            self.flow = flow
            self.reverse = None  # Reverse edge, linked later
    
    def add_edge(self, u: int, v: int, capacity: int) -> None:
        if any( edge.v == v for edge in self.graph[u]):
            raise ValueError(f"Duplicate edge from {u} to {v} is not allowed.")
        forward_edge = self.Edge(u, v, capacity)
        reverse_edge = self.Edge(v, u, 0)

        self.graph[u].append(forward_edge)
        self.graph[v].append(reverse_edge)
        

    def dfs(self, u: int, sink: int, flow: float) -> float:
        if u == sink:
            return flow  
        while self.ptr[u] < len(self.graph[u]):
            edge = self.graph[u][self.ptr[u]]

            if self.level[edge.v] == self.level[u] + 1 and edge.capacity - edge.flow > 0:
                pushed_flow = self.dfs(
                    edge.v, sink, min(flow, edge.capacity - edge.flow))
                
                if pushed_flow > 0:
                    edge.flow += pushed_flow
                    edge.reverse.flow -= pushed_flow
                    return pushed_flow      
            self.ptr[u] += 1
        return  0             


    def bfs(self, source, sink):
        # building level graph
        self.level = [-1] * self.n # reset levels
        self.level[source] = 0
        queue = deque([source]) # queue for BFS

        while queue:
            u = queue.popleft()
            for edge in self.graph[u]:
                 # block edges does not have any capacity any more
                if self.level[edge.v] == -1 and edge.capacity - edge.flow > 0:                
                    self.level[edge.v] = self.level[u] + 1
                    queue.append(edge.v)
        return self.level[sink] != -1 # if not reachable any more
    
        

    def dinic_max_flow(self, source, sink):
        total_flow = 0        
        # repeat until no augmenting path is found
        while self.bfs(source, sink):
            self.ptr = [0] * self.n  # Reset pointers for DFS
            while True:
                flow = self.dfs(source, sink, float('Inf'))
                if flow == 0:
                    break
                total_flow += flow
        return total_flow
    
    

